This notebook builds and outputs training data for use by Koopman-MPC learning and control algorithms. $M_t$ trajectories of length $T^j_t$ and initial conditions $x^j_0$ $\epsilon$ $\Omega$ are simulated, where $j=1,...,M_t$ and $\Omega$ denotes the set of all permissible states. From each trajectory, $M^j_s = (T^j_t/\Delta t)$ state and control actions are sampled at a fixed time interval $\Delta t$, resulting in the dataset:

$D = ((x^j_{k}, u^j_{k})^{M^j_s}_{k=1})^{M_t}_{j=1}$
 
This dataset is reformatted and output into the following data matrices for extended dynamic mode decomposition (eDMD):

$X = [x^1_1 \ ... \ x^1_{M^1_s-1} \ ... \ x^{M_t}_1 \ ... \ x^{M_t}_{M^{M_t}_s-1}]$

$Y = [x^1_1 \ ... \ x^1_{M^1_s} \ ... \ x^{M_t}_1 \ ... \ x^{M_t}_{M^{M_t}_s}]$

$U = [u^1_1 \ ... \ u^1_{M^1_s-1} \ ... \ u^{M_t}_1 \ ... \ u^{M_t}_{M^{M_t}_s-1}]$

Ideally, we wish to enrich our training data with as much information as possible so to maximize the ability of our DNNs to generalize to new data. This enrichment can mean e.g. different initialization positions, different trajectory properties (e.g. trajectory radii, linear accelerations, etc.), different trajectory types, variable peak velocities, and different unmodeled effects. For now, this code only implements loop trajectoies and aerodynamic drag as an unmodeled effect. We may also tactifully decide to withhold information from our training datasets so as to test the adaptation abilities of our trained Koopman-MPC DNN models.

Much of the code here is a replication of what's found in `track_circular_nominal.ipynb`, though that code has been adapted to streamline the execution of multiple simulations and produce training datasets as described. Note also that our implementation here is restricted by `loop_trajectory`, in that rather than being able to set a fixed $T_t$ and $M_s$ across all simulations, the number of data points and length of the simulation is predetermined by `traj_v_max` and `traj_a_lin`, and the fact that `loop_trajectory` forces a specific procedure for quadrotor operation (that is, acceleration to peak velocity followed by deceleration to standstill). In the future, `loop_trajectory` will be heavily refactored to allow for more flexibility, though this is not a high priority task at the moment as we believe that we can use the existing code to generate a rich enough training dataset (the most important thing to incorporate is variable peak velocities, as aerodynamic drag becomes a very important effect to model at high speeds).

Last note: certain combinations of `traj_v_max` and `traj_a_lin` fail to pass trajectory integrity tests as implemented in `check_trajectory`. It's not yet understood what `check_trajectory` is actually testing and why the aforementioned combinations are failing. Though I have noticed that past a certain value for `traj_a_lin` (~2.5), tests fail regardless of the values of `traj_v_max` and `traj_radius`.


COMMENT ON PADDING PROCEDURE AND WHY USED

# Configure settings

In [1]:
import numpy as np

Mt = 10 # number of trajectories

# toggle simulation disturbances

simulation_disturbances = {
    "drag": True,
}

# define quadrotor and world properties

m = 1.0 # quadrotor mass in [kg]
J = np.array([.03, .03, .06]) # quadrotor moment of inertia vector in [kg⋅m⋅m]
l = 0.235 # length between motor and quadrotor CoG in [m]
c = 0.013 # torque generated by each motor in direction of quadrotor z-axis in [N⋅m]
T_max = 10 # max thrust generated by each motor in [N]
g = 9.81 # gravitational acceleration of world in [m/s/s]

# define drag coefficients

rotor_drag = np.array([0.3, 0.3, 0.0])[:, np.newaxis] # rotor drag coefficients in [kg/m]
aero_drag = 0.08 # aerodynamic drag coefficient in [kg/m]

# define MPC and RK4 settings

t_horizon = 1.0 # prediction horizon in [s]
n_mpc_nodes = 10 # number of control nodes within horizon [?]
Q = np.array([10, 10, 10, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]) # weighing matrix for quadratic cost function
R = np.array([0.1, 0.1, 0.1, 0.1]) # weighing matrix for quadratic cost function
simulation_dt = 5e-4 # integration step size in [s]

# define trajectory settings

traj_radius = [1, 2, 3, 4, 5, 5, 5, 7, 10, 5] # radius of trajectory in [m]
traj_v_max = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20] # maximum speed at peak velocity in [m/s]
traj_a_lin = [0.25, 0.5, 0.75, 1, 2, 2, 2, 2, 2.5, 2.5] # linear acceleration of trajectory in [m/s/s]
reference_over_sampling = 5

# define initial positions of quadrotor

init_pos = [np.array([0.0, 0.0, 0.0]), np.array([2.0, 0.0, 0.0]), np.array([0.0, 0.0, 2.0]), 
            np.array([2.0, 2.0, 0.0]), np.array([2.0, 2.0, 2.0]), np.array([0.0, 0.0, 10.0]),
            np.array([0.0, 5.0, 5.0]), np.array([10.0, 2.0, 2.0]), np.array([10.0, 10.0, 10.0]),
            np.array([10.0, 2.0, 5.0])]

# perform assertions

assert Mt == len(traj_radius)
assert Mt == len(traj_v_max)
assert Mt == len(traj_a_lin)
assert Mt == len(init_pos)

# Perform additional setup

In [2]:
import sys

from quad import Quad
from quad_mpc_nominal import QuadMPCNominal
from trajectories import check_trajectory, loop_trajectory

# instantiate Quad and QuadMPC objects

quad = Quad(m=m, J=J, l=l, c=c, T_max=T_max, g=g, drag=simulation_disturbances["drag"], rotor_drag=rotor_drag, aero_drag=aero_drag)
quad_mpc = QuadMPCNominal(quad=quad, t_horizon=t_horizon, n_nodes=n_mpc_nodes, Q=Q, R=R)

# generate reference trajectory and control policy (burrowed entirely from https://github.com/uzh-rpg/data_driven_mpc/tree/main)

control_period = t_horizon / (n_mpc_nodes * reference_over_sampling) # sampling period of trajectory

reference_traj = {}; reference_timestamps = {}; reference_u = {}; reference_speed = {}

for i in range(Mt):
    dataset_name = f'dataset_{i+1}'
    reference_traj[dataset_name], reference_timestamps[dataset_name], reference_u[dataset_name] = loop_trajectory(
        quad=quad, discretization_dt=control_period, radius=traj_radius[i], lin_acc=traj_a_lin[i], clockwise=True, yawing=False, v_max=traj_v_max[i], plot=False)

    if not check_trajectory(reference_traj[dataset_name], reference_timestamps[dataset_name], plot=False):
        print("Reference trajectory integrity check failed!")
        sys.exit(1)
    
# compute reference speeds

for key, value in reference_traj.items():
    reference_speed[key] = np.linalg.norm(reference_traj[key][:,7:10], axis=1)

Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please export ACADOS_SOURCE_DIR to avoid this warning.
Please exp

ld: warning: directory not found for option '-L-l'


# Perform simulations

In [3]:
import os
import pickle

from datetime import datetime
from utils.math import separate_variables
from utils.mpc import get_reference_chunk

n_quad_states = len(quad.get_state())
n_control_states = len(quad.u)

data = {'X': np.empty((n_quad_states, 0)), 
        'Y': np.empty((n_quad_states, 0)), 
        'U': np.empty((n_control_states, 0)),
        'full_traj': np.empty((n_quad_states, 0)),
        'indices': []}

for index, (key, value) in enumerate(reference_traj.items()):

    n_data_pts = len(value)

    # initialize quadrotor state

    initial_state = np.split(value[0, :], [3, 7, 10])
    initial_state[0] = init_pos[index]
    quad.set_state(*initial_state)

    # begin simulation

    quad_traj = np.zeros([n_data_pts, n_quad_states]) # store quadrotor trajectory
    opt_u_history = np.zeros([n_data_pts-1, n_control_states]) # store optimal control action history
    tracking_error = np.zeros(n_data_pts) # store tracking error

    print("\nRunning MPC loop for simulation " + str(index) + " ...")

    for i in range(n_data_pts):
        
        # retrieve and save quadrotor state

        quad_current_state = quad.get_state()
        quad_traj[i, :] = np.expand_dims(quad_current_state, axis=0)

        # compute tracking error

        tracking_error[i] = np.linalg.norm(value[i, :3] - quad_traj[i, :3])

        if i < n_data_pts-1:

            # get the chunk of trajectory required for the current optimization (burrowed entirely from https://github.com/uzh-rpg/data_driven_mpc/tree/main)

            ref_traj_chunk, ref_u_chunk = get_reference_chunk(reference_traj[key], reference_u[key], i, n_mpc_nodes, reference_over_sampling)

            # set the reference for the OCP
        
            quad_mpc.set_reference(x_ref=separate_variables(ref_traj_chunk), u_ref=ref_u_chunk)

            # solve OCP to retrieve optimized control and state sequences
        
            u_opt_acados, x_opt_acados = quad_mpc.optimize()

            # select first set of control actions as the control to apply to the plant

            opt_u = u_opt_acados[:4]
            opt_u_history[i, :] = np.reshape(opt_u, (1, -1))
        
            # apply control action to quadrotor and update state

            simulation_time = 0.0
            while simulation_time < control_period:
                quad.update(opt_u, simulation_dt)
                simulation_time += simulation_dt

    # convert data to DMD-matrix format

    data['X'] = np.append(data['X'], quad_traj[0:-1].T, axis=1)
    data['Y'] = np.append(data['Y'], quad_traj[1:].T, axis=1)
    data['U'] = np.append(data['U'], opt_u_history[0:].T, axis=1)
    data['U'] = np.append(data['U'], np.full((n_control_states, 1), np.nan), axis=1)
    data['full_traj'] = np.append(data['full_traj'], quad_traj[0:].T, axis=1)

    if index == 0:
        data['indices'].append((0, n_data_pts-1))
    else:
        data['indices'].append((data['indices'][-1][1]+1, data['indices'][-1][1]+n_data_pts))

# save data file

data_dir = os.getcwd() + '/output/data/'
current_date = datetime.now().strftime('%m%d%Y')

def get_next_index(file_prefix, extension):
    index = 0
    while True:
        file_name = f'{file_prefix}_{index:02d}.{extension}'

        try:
            with open(file_name, 'rb'):
                index += 1
        except FileNotFoundError:
            break

    return index

file_prefix = data_dir + current_date
extension = 'pk1'

index = get_next_index(file_prefix, extension)

file_name = f'{file_prefix}_{index:02d}.{extension}'

with open(file_name, 'wb') as pickle_file:
    pickle.dump(data, pickle_file)


Running MPC loop for simulation 0 ...

Running MPC loop for simulation 1 ...

Running MPC loop for simulation 2 ...

Running MPC loop for simulation 3 ...

Running MPC loop for simulation 4 ...

Running MPC loop for simulation 5 ...

Running MPC loop for simulation 6 ...

Running MPC loop for simulation 7 ...

Running MPC loop for simulation 8 ...

Running MPC loop for simulation 9 ...
